In [1]:
import os.path
has_train_data = os.path.isfile('train.p') 
if not has_train_data:
    from zipfile import ZipFile
    import urllib.request
    from tempfile import mktemp

    zipurl = 'https://d17h27t6h515a5.cloudfront.net/topher/2016/November/581faac4_traffic-signs-data/traffic-signs-data.zip'
    from urllib.request import urlopen
    from zipfile import ZipFile
    zipresp = urlopen(zipurl)
    tempzip = open("/tmp/tempfile.zip", "wb")
    tempzip.write(zipresp.read())
    tempzip.close()
    zf = ZipFile("/tmp/tempfile.zip")
    zf.extractall()
    zf.close()

In [2]:
import pickle
train_data = 'train.p'
test_data = 'test.p'
with open(train_data, 'rb') as f:
    pickle_data = pickle.load(f)
    train_features = pickle_data['features']
    train_labels = pickle_data['labels']
    del pickle_data  # Free up memory
with open(test_data, 'rb') as f:
    pickle_data = pickle.load(f)
    test_features = pickle_data['features']
    test_labels = pickle_data['labels']
    del pickle_data  # Free up memory

In [3]:
import cv2
import numpy as np
gray_train_features = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in  train_features]
gray_test_features = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in  test_features]


In [4]:
def normalize_greyscale(image, a=0.1, b=0.9,grey_min=0,grey_max=255):
    return a + ( ( (image - grey_min)*(b - a) )/( grey_max - grey_min ) )
train_features = [normalize_greyscale(image) for image in gray_train_features]
test_features = [normalize_greyscale(image) for image in gray_test_features]

In [5]:
from sklearn.utils import shuffle
train_features, train_labels = shuffle(train_features,train_labels, random_state=0)

In [6]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
encoder.fit(train_labels)
train_labels = encoder.transform(train_labels)
test_labels = encoder.transform(test_labels)

In [7]:
# Change to float32, so it can be multiplied against the features in TensorFlow, which are float32
train_labels = train_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)

In [8]:
import tensorflow as tf

# Parameters
learning_rate = 0.01
training_epochs = 1000
batch_size = 100
display_step = 25


n_input = 1024
n_classes = 43


n_hidden_layer = 512



weights = {
    'hidden_layer': tf.Variable(tf.random_normal([n_input, n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_hidden_layer, n_classes]))
}
biases = {
    'hidden_layer': tf.Variable(tf.random_normal([n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# tf Graph input
x = tf.placeholder("float", [None, 32, 32])
y = tf.placeholder("float", [None, n_classes])

x_flat = tf.reshape(x, [-1, n_input])


# Hidden layer with RELU activation
layer_1 = tf.add(tf.matmul(x_flat, weights['hidden_layer']), biases['hidden_layer'])

relu_layer = tf.nn.relu(layer_1)
# Output layer with linear activation
logits = tf.matmul(relu_layer, weights['out']) + biases['out']

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)


In [9]:
# Initializing the variables
init = tf.initialize_all_variables()


# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(train_features)/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_start = i*batch_size
            batch_features = train_features[batch_start:batch_start + batch_size]
            batch_labels = train_labels[batch_start:batch_start + batch_size]
            #sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_features, y: batch_labels})
            # Compute average loss
            avg_cost += c / total_batch
            
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print ("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    print("Accuracy on testing data:", accuracy.eval({x: test_features, y: test_labels}))
    print("Accuracy on training data:", accuracy.eval({x: train_features, y: train_labels}))

Epoch: 0025 cost= 2.973548735
Epoch: 0050 cost= 1.446448641
Epoch: 0075 cost= 0.912382811
Epoch: 0100 cost= 0.644357568
Epoch: 0125 cost= 0.489814470
Epoch: 0150 cost= 0.388747719
Epoch: 0175 cost= 0.318847109
Epoch: 0200 cost= 0.267482427
Epoch: 0225 cost= 0.228703348
Epoch: 0250 cost= 0.199247634
Epoch: 0275 cost= 0.176201948
Epoch: 0300 cost= 0.157569854
Epoch: 0325 cost= 0.142311266
Epoch: 0350 cost= 0.129589173
Epoch: 0375 cost= 0.118905785
Epoch: 0400 cost= 0.109732456
Epoch: 0425 cost= 0.101721081
Epoch: 0450 cost= 0.094648091
Epoch: 0475 cost= 0.088365743
Epoch: 0500 cost= 0.082759696
Epoch: 0525 cost= 0.077720858
Epoch: 0550 cost= 0.073192247
Epoch: 0575 cost= 0.069108768
Epoch: 0600 cost= 0.065404368
Epoch: 0625 cost= 0.062040107
Epoch: 0650 cost= 0.058949228
Epoch: 0675 cost= 0.056140027
Epoch: 0700 cost= 0.053566911
Epoch: 0725 cost= 0.051198727
Epoch: 0750 cost= 0.048992261
Epoch: 0775 cost= 0.046947816
Epoch: 0800 cost= 0.045040838
Epoch: 0825 cost= 0.043246757
Epoch: 085